# Exploratory Data Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [ ]:
data = pd.read_csv('../data/opportunities.csv', parse_dates=['Create.Day'], encoding='latin1')

Drop any columns we're not going to be interested in:

-  Opportunity.Number 
-  Acquisition.Day 
-  Customer.Number 
-  End.Day 
-  Order.Number 
-  Order.End.Day 
-  Order.Start.Day 
-  Order.Type 
-  Start.Day 
-  Probability.x
-  Probability.y
-  Order.Entry.Weight.CHF
-  Order.Entry.Weight.EUR

All order related columns are a very clear of potential leakage. Once there's an order, than the opportunity is most likely already won or at least very close to being won.

As far as date fiels, our sales were are able to justify only the create day as the day when the opportunity is inserted into the CRM, so we'll leave the others for now.

Probabilities are just the very last one at the moment the opp was closed, and amounts weighted are just the product of the probability with the raw amount.

In [ ]:
drop_columns = ['Acquisition.Day', 'Customer.Number', 'End.Day', 'Order.Number', 'Order.End.Day', 
                'Order.Start.Day', 'Order.Type', 'Start.Day', 'Probability.x', 'Probability.y', 'Order.Entry.Weight.CHF',
               'Order.Entry.Weight.EUR']
data.drop(drop_columns, axis=1, inplace=True)

data['Order.Entry.CHF'] = data['Order.Entry.CHF'].map(lambda x: float(re.sub(',','',x)))
data['Order.Entry.EUR'] = data['Order.Entry.EUR'].map(lambda x: float(re.sub(',','',x)))
data['Year'] = data['Create.Day'].map(lambda x: x.year)

In [ ]:
data.drop(['Quantity'], axis=1, inplace=True)

Drop all samples were Customer is NULL

In [ ]:
data = data[data.Customer.notnull()]

In [ ]:
data.shape

## Response Variable

In [ ]:
data.Status.value_counts(dropna=False).sort_index()

In [ ]:
data['Status.Category'].value_counts(dropna=False).sort_index()

In [ ]:
data['Target'] = data['Status.Category'].map(lambda x: 1 if x=='Won' else 0)
np.mean(data.Target)

Almost 62% for majority class.

Classes are pretty unbalanced and also it is realistical to think that more opportunities are actually worked on, that they are won, so even a 50% proportion for the majority class would have been at the very least suspect.

For the moment I have no idea if I want to handle this and how.

## Customer features engineering

### First contact

Support column to facilitate aggregations and counts.

In [ ]:
data['one'] = 1

Before we can apply any logic that has to do with sorting with respect to the customer dimension, we should make sure records are alreay sorted

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data = data.sort_values(by=['Customer', 'Create.Day', 'Opportunity.Number'])

In [ ]:
data['Customer.Contacts.So.Far'] = data.groupby('Customer').cumsum()['one']

In [ ]:
data['Customer.First.Contact'] = data['Customer.Contacts.So.Far'].map(lambda x: 1 if x==1 else 0)

In [ ]:
data[['Customer', 'Create.Day', 'Customer.Contacts.So.Far', 
      'Customer.First.Contact']].sort_values(by = ['Customer', 'Create.Day']).head(20)

### How many so far

Customer.Contacts.So.Far

### Convertion rate so far

In [ ]:
data['Customer.Won.So.Far'] = data.groupby('Customer').cumsum()['Target']

In [ ]:
data['Customer.ConvRatio.So.Far'] = data['Customer.Won.So.Far'].astype('float') / data['Customer.Contacts.So.Far']

In [ ]:
data[['Customer', 'Create.Day', 'Target', 'Customer.Contacts.So.Far', 'Customer.Won.So.Far', 
      'Customer.ConvRatio.So.Far']].sort_values(by=['Customer','Create.Day']).head(20)

### Convertion rate last 5

In [ ]:
cvrl5 = data.groupby('Customer').rolling(window=5, on='Create.Day', min_periods=1).sum()['Target']

In [ ]:
cvrl5

In [ ]:
cvrl5 = cvrl5.reset_index()

In [ ]:
cvrl5

In [ ]:
cvrl5.drop('Customer', axis=1, inplace=True)

In [ ]:
cvrl5.rename(columns={'Target': 'Customer.Won.Last5', 'level_1': 'Opportunity.Number'}, inplace=True)

In [ ]:
cvrl5

In [ ]:
data = data.reset_index()

In [ ]:
data = data.merge(cvrl5, on='Opportunity.Number', how='left')

In [ ]:
data[['Customer', 'Create.Day', 'Target', 'Customer.Won.Last5']].sort_values(by=['Customer','Create.Day', 'Opportunity.Number']).head(20)

In [ ]:
data['Customer.ConvRatio.Last5'] = data['Customer.Won.Last5'] / data['Customer.Contacts.So.Far'].map(lambda x: min(x, 5))

In [ ]:
data[['Customer', 'Opportunity.Number', 'Create.Day', 'Target', 'Customer.Won.Last5', 
      'Customer.ConvRatio.Last5']].sort_values(by=['Customer','Create.Day']).head(20)

### How many in last year, convertion ratio last year?

In [ ]:
import datetime

In [ ]:
cvrl5 = data.groupby('Customer', ).rolling(window=datetime.timedelta(days=365), 
                                           on='Create.Day', min_periods=1).sum()[['Target', 'one']]
cvrl5 = cvrl5.reset_index()
cvrl5.drop('Customer', axis=1, inplace=True)
cvrl5.rename(columns={'Target': 'Customer.Won.LastYear', 'one': 'Customer.Contacts.LastYear', 
                     'level_1': 'Opportunity.Number'}, inplace=True)

In [ ]:
data = data.reset_index()
data = data.merge(cvrl5, on='Opportunity.Number', how='left')
data.set_index('Opportunity.Number', inplace=True)

In [ ]:
data[['Customer', 'Create.Day', 'Target', 'Customer.Won.LastYear']].sort_values(by=['Customer','Create.Day']).head(20)

In [ ]:
data['Customer.ConvRatio.LastYear'] = data['Customer.Won.LastYear'] / data['Customer.Contacts.LastYear']

In [ ]:
data[['Customer', 'Create.Day', 'Target', 'Customer.Won.LastYear', 
      'Customer.Contacts.LastYear', 'Customer.ConvRatio.LastYear']].sort_values(by=['Customer','Create.Day']).head(30)

### Standard deviation of current Order Entry CHF to average value for a given customer

In [ ]:
data['Order.Entry.CHF.Won'] = data['Order.Entry.CHF'] * data['Target']

In [ ]:
data['Customer.Avg.Order.Entry.CHF.So.Far'] = data.groupby('Customer').cumsum()['Order.Entry.CHF.Won'] / data['Customer.Won.So.Far']

In [ ]:
data[['Customer', 'Create.Day', 'Target', 'Order.Entry.CHF', 'Order.Entry.CHF.Won',
      'Customer.Contacts.So.Far', 'Customer.Avg.Order.Entry.CHF.So.Far']].sort_values(by=['Customer', 'Create.Day']).head(20)

In [ ]:
data['Customer.Avg.Order.Entry.CHF.So.Far.Lag1'] = data.groupby('Customer')['Customer.Avg.Order.Entry.CHF.So.Far'].shift(1)

In [ ]:
data[['Customer', 'Create.Day', 'Customer.Avg.Order.Entry.CHF.So.Far', 
      'Customer.Avg.Order.Entry.CHF.So.Far.Lag1']].sort_values(by=['Customer', 'Create.Day']).head(20)

In [ ]:
data['Customer.Order.Entry.CHF.std2avg'] = (data['Order.Entry.CHF'] - data['Customer.Avg.Order.Entry.CHF.So.Far.Lag1']) /  \
                    data['Customer.Avg.Order.Entry.CHF.So.Far.Lag1'].map(lambda x: np.nan if x==0 else x)

In [ ]:
data[['Customer', 'Create.Day', 'Target', 'Order.Entry.CHF',
      'Customer.Avg.Order.Entry.CHF.So.Far.Lag1', 'Customer.Order.Entry.CHF.std2avg']].sort_values(by=['Customer', 'Create.Day']).head(20)

In [ ]:
data['Customer.Order.Entry.CHF.std2avg'] = data['Customer.Order.Entry.CHF.std2avg'].fillna(0)

In [ ]:
data.groupby('Target').mean()['Customer.Order.Entry.CHF.std2avg']

### Last outcome

In [ ]:
data['Customer.Last.Target'] = data.groupby('Customer')['Target'].shift(1)

### Time (days) since last contact

In [ ]:
data['Customer.Days.Since.LastContact'] = (data['Create.Day'] - data.groupby('Customer')['Create.Day'].shift(1)).dt.days

### Time (days) since last win

In [ ]:
idxWon = (data.Target == 1)
data.loc[idxWon, 'Customer.Last.Day.Won'] = data.loc[idxWon, 'Create.Day']
data['Customer.Last.Day.Won'] = data['Customer.Last.Day.Won'].fillna(method='ffill')
data['Customer.Last.Day.Won'] = data['Customer.Last.Day.Won'].shift(1)
data['Customer.Days.Since.LastWin'] = (data['Create.Day'] - data['Customer.Last.Day.Won']).dt.days

What are negative values?

### Time (days) since last loss

In [ ]:
idxLost = (data.Target == 0)
data.loc[idxWon, 'Customer.Last.Day.Loss'] = data.loc[idxLost, 'Create.Day']
data['Customer.Last.Day.Loss'] = data['Customer.Last.Day.Loss'].fillna(method='ffill')
data['Customer.Last.Day.Loss'] = data['Customer.Last.Day.Loss'].shift(1)
data['Customer.Days.Since.LastLoss'] = (data['Create.Day'] - data['Customer.Last.Day.Loss']).dt.days

### Amount CHF last win

In [ ]:
idxWon = (data.Target == 1)
data.loc[idxWon, 'Customer.CHF.Last.Won'] = data.loc[idxWon, 'Order.Entry.CHF']
data['Customer.CHF.Last.Won'] = data['Customer.CHF.Last.Won'].fillna(method='ffill')
data['Customer.CHF.Last.Won'] = data['Customer.CHF.Last.Won'].shift(1)

### Amount  CHF last loss

In [ ]:
idxLoss = (data.Target == 0)
data.loc[idxWon, 'Customer.CHF.Last.Loss'] = data.loc[idxLoss, 'Order.Entry.CHF']
data['Customer.CHF.Last.Loss'] = data['Customer.CHF.Last.Loss'].fillna(method='ffill')
data['Customer.CHF.Last.Loss'] = data['Customer.CHF.Last.Loss'].shift(1)

In [ ]:
data.info()

In [ ]:
data.to_csv('../data/test.csv', index=False, encoding='latin1')